In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext watermark

import os
from io import StringIO

import numpy as np
import pandas as pd
import requests
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
DATA_LOCATION = '/mnt/dataA/TCGA/processed/'

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#miRNA" data-toc-modified-id="miRNA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>miRNA</a></span><ul class="toc-item"><li><span><a href="#Map-files-to-patients" data-toc-modified-id="Map-files-to-patients-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Map files to patients</a></span><ul class="toc-item"><li><span><a href="#Drop-unused-patients" data-toc-modified-id="Drop-unused-patients-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Drop unused patients</a></span></li></ul></li><li><span><a href="#Load-all-data" data-toc-modified-id="Load-all-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load all data</a></span></li><li><span><a href="#Scale-values" data-toc-modified-id="Scale-values-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Scale values</a></span></li><li><span><a href="#Save-to-files" data-toc-modified-id="Save-to-files-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Save to files</a></span></li></ul></li><li><span><a href="#mRNA" data-toc-modified-id="mRNA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>mRNA</a></span><ul class="toc-item"><li><span><a href="#Map-files-to-patients" data-toc-modified-id="Map-files-to-patients-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Map files to patients</a></span><ul class="toc-item"><li><span><a href="#Drop-unused-patients" data-toc-modified-id="Drop-unused-patients-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Drop unused patients</a></span></li></ul></li><li><span><a href="#Pick-gene-subset" data-toc-modified-id="Pick-gene-subset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Pick gene subset</a></span><ul class="toc-item"><li><span><a href="#Compute-variance-for-all-genes" data-toc-modified-id="Compute-variance-for-all-genes-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Compute variance for all genes</a></span></li><li><span><a href="#Define-gene-subset" data-toc-modified-id="Define-gene-subset-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Define gene subset</a></span></li></ul></li><li><span><a href="#Load-all-data" data-toc-modified-id="Load-all-data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Load all data</a></span></li><li><span><a href="#Scale-values" data-toc-modified-id="Scale-values-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Scale values</a></span></li><li><span><a href="#Save-to-files" data-toc-modified-id="Save-to-files-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Save to files</a></span></li></ul></li><li><span><a href="#DNA-methylation" data-toc-modified-id="DNA-methylation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>DNA methylation</a></span><ul class="toc-item"><li><span><a href="#Map-files-to-patients" data-toc-modified-id="Map-files-to-patients-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Map files to patients</a></span><ul class="toc-item"><li><span><a href="#Drop-unused-patients" data-toc-modified-id="Drop-unused-patients-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Drop unused patients</a></span></li></ul></li><li><span><a href="#Pick-probe-subset" data-toc-modified-id="Pick-probe-subset-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Pick probe subset</a></span><ul class="toc-item"><li><span><a href="#Save-to-file" data-toc-modified-id="Save-to-file-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Save to file</a></span></li><li><span><a href="#Compute-variance-for-all-selected-probes" data-toc-modified-id="Compute-variance-for-all-selected-probes-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Compute variance for all selected probes</a></span></li><li><span><a href="#Define-probe-subset" data-toc-modified-id="Define-probe-subset-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Define probe subset</a></span></li></ul></li><li><span><a href="#Load-all-data" data-toc-modified-id="Load-all-data-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load all data</a></span></li><li><span><a href="#Replace-nan-by-median-value" data-toc-modified-id="Replace-nan-by-median-value-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Replace <code>nan</code> by median value</a></span></li><li><span><a href="#Save-to-files" data-toc-modified-id="Save-to-files-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Save to files</a></span></li></ul></li><li><span><a href="#CNV" data-toc-modified-id="CNV-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>CNV</a></span><ul class="toc-item"><li><span><a href="#Map-aliquotes-to-patients" data-toc-modified-id="Map-aliquotes-to-patients-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Map aliquotes to patients</a></span></li><li><span><a href="#Load-all-data" data-toc-modified-id="Load-all-data-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Load all data</a></span></li><li><span><a href="#Are-there-unchanged-genes?" data-toc-modified-id="Are-there-unchanged-genes?-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Are there unchanged genes?</a></span></li><li><span><a href="#Replace-aliquote-by-patient-IDs" data-toc-modified-id="Replace-aliquote-by-patient-IDs-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Replace aliquote by patient IDs</a></span><ul class="toc-item"><li><span><a href="#Check-duplicate-patient-IDs" data-toc-modified-id="Check-duplicate-patient-IDs-4.4.1"><span class="toc-item-num">4.4.1&nbsp;&nbsp;</span>Check duplicate patient IDs</a></span></li></ul></li><li><span><a href="#Encode-categorical-variables" data-toc-modified-id="Encode-categorical-variables-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Encode categorical variables</a></span><ul class="toc-item"><li><span><a href="#Drop-unused-patients" data-toc-modified-id="Drop-unused-patients-4.5.1"><span class="toc-item-num">4.5.1&nbsp;&nbsp;</span>Drop unused patients</a></span></li></ul></li><li><span><a href="#Keep-only-high-variance-genes" data-toc-modified-id="Keep-only-high-variance-genes-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Keep only high-variance genes</a></span></li><li><span><a href="#Save-to-files" data-toc-modified-id="Save-to-files-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Save to files</a></span></li></ul></li></ul></div>

# Functions <a class='tocSkip'></a>

In [ ]:
def request_file_info(data_type):
    fields = [
        'file_name',
        'cases.submitter_id',
        'cases.samples.sample_type',
        'cases.project.project_id',
        'cases.project.primary_site',
        ]

    fields = ','.join(fields)

    files_endpt = 'https://api.gdc.cancer.gov/files'

    filters = {
        'op': 'and',
        'content':[
            {
            'op': 'in',
            'content':{
                'field': 'files.experimental_strategy',
                'value': [data_type]
                }
            }
        ]
    }

    params = {
        'filters': filters,
        'fields': fields,
        'format': 'TSV',
        'size': '200000'
        }

    response = requests.post(files_endpt, headers = {'Content-Type': 'application/json'}, json = params)

    return pd.read_csv(StringIO(response.content.decode('utf-8')), sep='\t')

In [ ]:
def make_patient_file_map(df, base_dir):
    d = {}
    
    for _, row in df.iterrows():
        patient = row['cases.0.submitter_id']
        if patient in d:
            if not isinstance(d[patient], tuple):
                d[patient] = (
                    d[patient],
                    os.path.join(base_dir, row.id, row.file_name))
            else:
                d[patient] += os.path.join(base_dir, row.id, row.file_name),
        else:
            d[patient] = os.path.join(base_dir, row.id, row.file_name)

    return d

In [ ]:
def min_max_scale(data, features, groups):
    train = data.loc[data.index.isin(groups['train']), features]

    scaler = MinMaxScaler()
    columns = train.columns
    scaler = scaler.fit(train[columns])
    
    data.loc[data.index.isin(groups['train']), features] = scaler.transform(
        train)
    data.loc[data.index.isin(groups['val']), features] = scaler.transform(
        data.loc[data.index.isin(groups['val']), features])
    data.loc[data.index.isin(groups['test']), features] = scaler.transform(
        data.loc[data.index.isin(groups['test']), features])
    
    return data

In [ ]:
def table_to_patient_files(table, dir_path, round_digits=4):
    n = len(table)
    
    i = 0

    for index, row in table.iterrows():
        print('\r' + f'Save data to files: {str(i + 1)}/{n}', end='')
        i+= 1

        target_file = os.path.join(dir_path, str(index) + '.tsv')
        
        with open(target_file, 'w') as f:
            if round_digits is not None:
                f.write('\n'.join(str(round(value, round_digits)) for value in row.values))
            else:
                f.write('\n'.join(str(value) for value in row.values))

    print()
    print()

In [ ]:
def merge_all_tables(table_list):
    n = len(table_list)
    
    final_table = pd.DataFrame()

    for i, table in enumerate(table_list):
        print('\r' + f'Merge data tables: {str(i + 1)}/{n}', end='')

        if final_table.empty:
            final_table = table
        else:
            final_table = final_table.join(table)
    
    print()
    print('Final table dimensions:', final_table.shape)
    print()

    return final_table

In [ ]:
labels = pd.read_csv('../labels.tsv', sep='\t')
labels.head()

In [ ]:
id_groups = {
    'train': list(labels.loc[labels['group'] == 'train', 'submitter_id']),
    'val': list(labels.loc[labels['group'] == 'val', 'submitter_id']),
    'test': list(labels.loc[labels['group'] == 'test', 'submitter_id'])}

# miRNA

## Map files to patients

Use the [GDC API](https://docs.gdc.cancer.gov/API/Users_Guide/Python_Examples/#using-python-to-query-the-gdc-api) to retrieve mapping between file names and patient IDs. Collect results as Pandas `DataFrame`.

In [ ]:
%%time

miRNA_files = request_file_info(data_type='miRNA-Seq')
miRNA_files.shape

In [ ]:
miRNA_files.head()

In [ ]:
miRNA_files = miRNA_files[miRNA_files['cases.0.project.project_id'].str.startswith('TCGA')]
miRNA_files = miRNA_files[miRNA_files['file_name'].str.endswith('mirbase21.mirnas.quantification.txt')]
miRNA_files = miRNA_files[miRNA_files['cases.0.samples.0.sample_type'] == 'Primary Tumor']
miRNA_files.shape

Some cases have more than one `*mirbase21.mirnas.quantification.txt` file. I suppose these are replicate runs for the same patient. Accordingly, checking some examples showed that `reads_per_million_miRNA_mapped` values were highly correlated. Here I will simply keep the first one in the table and drop the other ones. This decision may be revisited later on if deemed useful.

In [ ]:
print('All rows:       ', miRNA_files.shape[0])
print('Unique patients:', miRNA_files['cases.0.submitter_id'].unique().shape[0])

In [ ]:
miRNA_files = miRNA_files[~miRNA_files.duplicated(subset=['cases.0.submitter_id'], keep='first')]
miRNA_files.shape

In [ ]:
file_map = make_patient_file_map(df=miRNA_files, base_dir='/mnt/dataA/TCGA/raw/miRNA-seq/')

### Drop unused patients

Keep only patients present in label data.

In [ ]:
len(file_map)

In [ ]:
labels = pd.read_csv('../data/labels.tsv', sep='\t')
len(labels['submitter_id'])

In [ ]:
file_map = {k: file_map[k] for k in file_map if k in list(labels['submitter_id'])}

In [ ]:
len(file_map)

## Load all data

In [ ]:
def load_all_miRNA_data(patient_file_map):
    n = len(patient_file_map)
    
    dfs = []
    for i, patient in enumerate(patient_file_map):
        print('\r' + f'Load data tables: {str(i + 1)}/{n}', end='')
        df = pd.read_csv(patient_file_map[patient], sep='\t',
                         usecols=['miRNA_ID', 'reads_per_million_miRNA_mapped'],
                         index_col=0)
        df.columns = [patient]
        dfs.append(df)

    print()
    print()

    return dfs

In [ ]:
%%time

data_tables = load_all_miRNA_data(patient_file_map=file_map)

In [ ]:
%%time

miRNA = merge_all_tables(data_tables)

## Scale values

In [ ]:
miRNA.shape

In [ ]:
miRNA.head()

In [ ]:
miRNA = min_max_scale(data=miRNA.T, features=miRNA.T.columns, groups=id_groups)

In [ ]:
miRNA.head()

In [ ]:
miRNA.loc[miRNA.index.isin(['TCGA-A8-A09K', 'TCGA-EW-A1OX', 'TCGA-A2-A3KD'])]

## Save to files

# mRNA

## Map files to patients

Use the [GDC API](https://docs.gdc.cancer.gov/API/Users_Guide/Python_Examples/#using-python-to-query-the-gdc-api) to retrieve mapping between file names and patient IDs. Collect results as Pandas `DataFrame`.

In [ ]:
%%time

mRNA_files = request_file_info(data_type='RNA-Seq')
mRNA_files.shape

In [ ]:
mRNA_files.head()

In [ ]:
mRNA_files = mRNA_files[mRNA_files['cases.0.project.project_id'].str.startswith('TCGA')]
mRNA_files = mRNA_files[mRNA_files['file_name'].str.endswith('FPKM-UQ.txt.gz')]
mRNA_files = mRNA_files[mRNA_files['cases.0.samples.0.sample_type'] == 'Primary Tumor']
mRNA_files.shape

As for miRNA above, when there is more than one file for a single patient just keep the first one. Again, this decision may be revisited later on if deemed useful.

In [ ]:
print('All rows:       ', mRNA_files.shape[0])
print('Unique patients:', mRNA_files['cases.0.submitter_id'].unique().shape[0])

In [ ]:
mRNA_files = mRNA_files[~mRNA_files.duplicated(subset=['cases.0.submitter_id'], keep='first')]
mRNA_files.shape

In [ ]:
file_map = make_patient_file_map(mRNA_files, base_dir='/mnt/dataA/TCGA/raw/RNA-seq_FPKM-UQ/')

### Drop unused patients

Keep only patients present in label data.

In [ ]:
len(file_map)

In [ ]:
labels = pd.read_csv('../data/labels.tsv', sep='\t')
len(labels['submitter_id'])

In [ ]:
file_map = {k: file_map[k] for k in file_map if k in list(labels['submitter_id'])}

In [ ]:
len(file_map)

## Pick gene subset

Most genes are probably not informative. Check variance of each gene across patients and remove low-variance genes to reduce data size.

Calculate variance iteratively for gene subsets, to avoid memory issues.

### Compute variance for all genes

Run dedicated script (found here in `src/scripts` directory) as follows:

```bash
$ conda activate ig
$ python ./compute_gene_variance.py \
   -i /mnt/dataA/TCGA/raw/RNA-seq_FPKM-UQ/
   -s 10000 \
   -o /mnt/dataA/TCGA/raw/RNA-seq_FPKM-UQ_variance.tsv
```

### Define gene subset

Keep a sensible number of genes showing the highest variance across patients.

Running the model on mRNA data after keeping increasing numbers of genes (500, 750, 1'000, or 2'000) showed slight increases in training data C-index (~2 percentage points). Picked the subset yielding highest validation C-index.

In [ ]:
# Load computed variance table
variance_table = pd.read_csv('/mnt/dataA/TCGA/raw/RNA-seq_count_variance.tsv', sep='\t',
                             header=None, index_col=0, names=['Var'])

In [ ]:
print(f'# of represented mRNAs: {len(variance_table)}')

In [ ]:
variance_table.head()

In [ ]:
variance_table.sort_values('Var', ascending=False).head(10)

In [ ]:
variance_table.describe()

In [ ]:
variance_table.boxplot()

In [ ]:
e = 1e-7
np.log10(variance_table + e).boxplot()

In [ ]:
len(variance_table[variance_table['Var'] > 1e9])

In [ ]:
variance_table[variance_table['Var'] > 1e9].sort_values(by='Var', ascending=False).plot(use_index=False)

In [ ]:
# Variance quantile to drop all genes but the top 2k
q = (variance_table.shape[0] - 2e3) / variance_table.shape[0]
print(f'Percentile = {round(q * 100, 1)}%')

variance_table[variance_table['Var'] > variance_table['Var'].quantile(q=q)].shape

In [ ]:
# Variance quantile to drop all genes but the top 1k
q = (variance_table.shape[0] - 1e3) / variance_table.shape[0]
print(f'Percentile = {round(q * 100, 1)}%')

variance_table[variance_table['Var'] > variance_table['Var'].quantile(q=q)].shape

In [ ]:
# Get IDs for all genes to keep
genes_to_keep = list(variance_table[variance_table['Var'] > variance_table['Var'].quantile(q=q)].index)
genes_to_keep[:5]

In [ ]:
len(genes_to_keep)

## Load all data

In [ ]:
def load_all_mRNA_data(patient_file_map, genes):
    n = len(patient_file_map)
    
    dfs = []
    for i, patient in enumerate(patient_file_map):
        print('\r' + f'Load data tables: {str(i + 1)}/{n}', end='')
        df = pd.read_csv(patient_file_map[patient], sep='\t', header=None,
                         index_col=0)
        df = df.loc[genes]
        df.columns = [patient]
        dfs.append(df)

    print()
    print()

    return dfs

In [ ]:
%%time

data_tables = load_all_mRNA_data(patient_file_map=file_map, genes=genes_to_keep)

In [ ]:
data_tables[0].head()

In [ ]:
%%time

mRNA = merge_all_tables(data_tables)

## Scale values

In [ ]:
mRNA.shape

In [ ]:
mRNA.head()

In [ ]:
mRNA = min_max_scale(data=mRNA.T, features=mRNA.T.columns, groups=id_groups)

In [ ]:
mRNA.head()

## Save to files

# DNA methylation

As explained in the GDC [Methylation Liftover Pipeline page](https://docs.gdc.cancer.gov/Data/Bioinformatics_Pipelines/Methylation_LO_Pipeline/), data was generated using either Illumina Infinium Human Methylation 27 (HM27; 27'578 probes) or HumanMethylation450 (HM450; 485'577 probes).

Use the intersection of the probes between the two (25'978 probes).

## Map files to patients

Use the [GDC API](https://docs.gdc.cancer.gov/API/Users_Guide/Python_Examples/#using-python-to-query-the-gdc-api) to retrieve mapping between file names and patient IDs. Collect results as Pandas `DataFrame`.

In [ ]:
%%time

DNAm_files = request_file_info(data_type='Methylation Array')
DNAm_files.shape

In [ ]:
DNAm_files.head()

In [ ]:
DNAm_files = DNAm_files[DNAm_files['cases.0.project.project_id'].str.startswith('TCGA')]
DNAm_files = DNAm_files[DNAm_files['file_name'].str.endswith('gdc_hg38.txt')]
DNAm_files = DNAm_files[DNAm_files['cases.0.samples.0.sample_type'] == 'Primary Tumor']
DNAm_files.shape

As for miRNA above, when there is more than one file for a single patient just keep the first one. Again, this decision may be revisited later, if deemed useful.

In [ ]:
print('All rows:       ', DNAm_files.shape[0])
print('Unique patients:', DNAm_files['cases.0.submitter_id'].unique().shape[0])

In [ ]:
DNAm_files = DNAm_files[~DNAm_files.duplicated(subset=['cases.0.submitter_id'], keep='first')]
DNAm_files.shape

In [ ]:
file_map = make_patient_file_map(DNAm_files, base_dir='/mnt/dataA/TCGA/raw/Methylation/')

### Drop unused patients

Keep only patients present in label data.

In [ ]:
len(file_map)

In [ ]:
labels = pd.read_csv('../data/labels.tsv', sep='\t')
len(labels['submitter_id'])

In [ ]:
file_map = {k: file_map[k] for k in file_map if k in list(labels['submitter_id'])}

In [ ]:
len(file_map)

## Pick probe subset

Check a small subset of 100 files to get an idea of the types of data and their proportions in the dataset.

In [ ]:
%%time

# List number of probes per file
total_n_lines = []
i = 0

for patient in file_map:
    i += 1
    if i > 100:
        break
    if isinstance(file_map[patient], tuple):
        total_n_lines = [total_n_lines.append(len(list(pd.read_csv(
            f, sep='\t', index_col=0, usecols=['Composite Element REF', 'Beta_value']).index)))
                         for f in file_map[patient]]
    else:
        total_n_lines.append(len(list(pd.read_csv(
            file_map[patient], sep='\t', index_col=0,
            usecols=['Composite Element REF', 'Beta_value']).index)))

In [ ]:
set(total_n_lines)

In [ ]:
element = 27578
total_n_lines.count(element)

In [ ]:
element = 485577
total_n_lines.count(element)

In [ ]:
eg_file = file_map[list(file_map.keys())[100]]
total_n_lines = len(list(pd.read_csv(
    eg_file, sep='\t', index_col=0, usecols=['Composite Element REF', 'Beta_value']).index))
total_n_lines

In [ ]:
DNAm = pd.read_csv(eg_file, sep='\t', index_col=0, usecols=['Composite Element REF', 'Beta_value'])

In [ ]:
DNAm.head()

In [ ]:
DNAm.shape

In [ ]:
probes = list(DNAm.index)
probes[:5]

In [ ]:
DNAm = pd.read_csv('/mnt/dataA/TCGA/raw/Methylation/0000c40e-9d45-4446-9dd9-a4676224d0ce/jhu-usc.edu_GBM.HumanMethylation450.7.lvl-3.TCGA-19-5955-01A-11D-1697-05.gdc_hg38.txt',
                   sep='\t', index_col=0, usecols=['Composite Element REF', 'Beta_value'])

In [ ]:
DNAm.head()

In [ ]:
DNAm.shape

In [ ]:
DNAm[DNAm.index == 'cg00000292']

In [ ]:
%%time

print('# Illumina Infinium Human Methylation 27 (HM27) probes found in HumanMethylation450 (HM450):')
print(sum([p in probes for p in list(DNAm.index)]))
print()

Use the intersection of the probe sets in the two technologies (25'978 according to the two example files used here).

In [ ]:
probe_set = [p for p in list(DNAm.index) if p in probes]

In [ ]:
len(probe_set)

In [ ]:
probe_set[:10]

### Save to file

In [ ]:
target_file = '../data/DNAm_probes.tsv'

In [ ]:
pd.read_csv(target_file, sep='\t', header=None, names=['Probes']).head()

In [ ]:
pd.read_csv(target_file, sep='\t', header=None, names=['Probes']).shape

### Compute variance for all selected probes

Run dedicated script (found here in `src/scripts` directory) as follows:

```bash
$ conda activate ig
$ python ./compute_DNAm_variance.py \
   -i /mnt/dataA/TCGA/raw/Methylation/ \
   -s 5000 \
   -l ../data/labels.tsv \
   -p ../data/DNAm_probes.tsv \
   -o /mnt/dataA/TCGA/raw/DNA_methylation_Beta-value_variance.tsv
```

### Define probe subset

Keep a sensible number of probes showing the highest variance across patients.

In [ ]:
# Load computed variance table
variance_table = pd.read_csv('/mnt/dataA/TCGA/raw/DNA_methylation_Beta-value_variance.tsv', sep='\t',
                             header=None, index_col=0, names=['Var'])

In [ ]:
variance_table.head()

In [ ]:
variance_table.sort_values('Var', ascending=False).head(10)

In [ ]:
variance_table.describe()

In [ ]:
variance_table.boxplot()

In [ ]:
e = 1e-7
np.log10(variance_table + e).boxplot()

In [ ]:
variance_table[variance_table['Var'] > 1e-2].sort_values(by='Var', ascending=False).plot(use_index=False)

In [ ]:
# Variance quantile to drop all genes but the chosen number of probes showing the top variance
# (since there are quite a lot of missing values, the result is lower than the selected number of probes)
q = (variance_table.shape[0] - 4e3) / variance_table.shape[0]
print(f'Percentile = {round(q * 100, 1)}%')

variance_table[variance_table['Var'] > variance_table['Var'].quantile(q=q)].shape

In [ ]:
# Get IDs for all probes to keep
probes_to_keep = list(variance_table[variance_table['Var'] > variance_table['Var'].quantile(q=q)].index)
probes_to_keep[:5]

In [ ]:
len(probes_to_keep)

## Load all data

In [ ]:
def load_all_DNAm_data(patient_file_map, probes):
    n = len(patient_file_map)
    
    dfs = []
    for i, patient in enumerate(patient_file_map):
        print('\r' + f'Load data tables: {str(i + 1)}/{n}', end='')
        df = pd.read_csv(patient_file_map[patient], sep='\t', index_col=0,
                         usecols=['Composite Element REF', 'Beta_value'])
        df = df.loc[probes]
        df.columns = [patient]
        dfs.append(df)

    print()
    print()

    return dfs

In [ ]:
%%time

data_tables = load_all_DNAm_data(patient_file_map=file_map, probes=probes_to_keep)

In [ ]:
data_tables[0].head()

In [ ]:
%%time

DNAm = merge_all_tables(data_tables)

__Note:__ There is <font style="color:red">no need to scale DNA methylation Beta Values</font> here, since they are already in the interval [0, 1].

In [ ]:
DNAm.shape

In [ ]:
DNAm.head()

In [ ]:
DNAm=DNAm.T

In [ ]:
DNAm.head()

## Replace `nan` by median value

In [ ]:
DNAm.isna().sum()

In [ ]:
DNAm.median()

In [ ]:
DNAm.fillna(DNAm.median()).isna().sum()

In [ ]:
DNAm = DNAm.fillna(DNAm.median())

## Save to files

# CNV

## Map aliquotes to patients

Use the [GDC API](https://docs.gdc.cancer.gov/API/Users_Guide/Python_Examples/#using-python-to-query-the-gdc-api) to retrieve mapping of aliquote IDs to patients found in the 33 tables (one per cancer entity). Collect results as Pandas `DataFrame`.

In [ ]:
%%time

response = requests.get('https://api.gdc.cancer.gov/cases',
                        params={
                            'fields': 'cases.samples.portions.analytes.aliquots',
                            'format': 'TSV',
                            'size': '200000'
                        })
cnv_aliquotes = pd.read_csv(StringIO(response.content.decode("utf-8")), sep="\t")

In [ ]:
cnv_aliquotes = cnv_aliquotes.set_index('submitter_id')
cnv_aliquotes.shape

In [ ]:
cnv_aliquotes.head()

In [ ]:
cnv_aliquotes = cnv_aliquotes[cnv_aliquotes.index.str.startswith('TCGA')]

In [ ]:
print('Unique patients:', cnv_aliquotes.index.unique().shape[0])

In [ ]:
cnv_aliquotes.index.duplicated

In [ ]:
aliquote_map = {}

aliquote_cols = [col for col in cnv_aliquotes if col.startswith('aliquot_ids')]

for index, row in cnv_aliquotes.iterrows():
    if index in aliquote_map.keys():
        print(index, 'already in aliquote map!')
    
    aliquote_map[index] = []
    row = row.loc[~row.isnull()]
    cols = [col for col in aliquote_cols if col in row]
    
    for col in cols:
        aliquote_map[index].append(row[col])

In [ ]:
len(aliquote_map)

## Load all data

In [ ]:
!ls /mnt/dataA/TCGA/raw/CNV/ | wc -l

In [ ]:
def load_all_CNV_data():
    base_dir = '/mnt/dataA/TCGA/raw/CNV/'
    project_dirs = [(os.path.join(base_dir, d)) for d in os.listdir(base_dir)]
    files = [(os.path.join(pd, f)) for pd in project_dirs
             for f in os.listdir(pd) if f.endswith('focal_score_by_genes.txt')]    
    
    dfs = []
    n = len(files)

    for i, f in enumerate(files):
        print('\r' + f'Load data tables: {str(i + 1)}/{n}', end='')
        df = pd.read_csv(f, sep='\t', index_col=0)
        df = df.iloc[:, 2:]
        dfs.append(df)

    print()
    print()

    return dfs

In [ ]:
data_tables = load_all_CNV_data()

In [ ]:
data_tables[0].head()

In [ ]:
cnv = merge_all_tables(data_tables)

In [ ]:
cnv.shape

## Are there unchanged genes?

Any genes with the same value for all patients will be completely uninformative.

In [ ]:
# Any genes with same CNV value across all samples?
unchanged_genes = []

for index, row in cnv.iterrows():
    if len(pd.unique(row)) < 2:
        unchanged_genes.append(index)

In [ ]:
print('# of unchanged genes:', len(unchanged_genes))

## Replace aliquote by patient IDs

In [ ]:
def lookup_submitter_id(df, aliquote_id_map):
    cols = df.columns
    n = len(cols)

    for i, col in enumerate(cols):
        print('\r' + f'Replace col names by patient IDs: {str(i + 1)}/{n}', end='')
        for patient in aliquote_id_map:
            if col in aliquote_id_map[patient]:
                patient_id = patient
            
        df.rename(columns={col: patient_id}, inplace=True)
   
    return df

In [ ]:
%%time

cnv = lookup_submitter_id(cnv, aliquote_map)
print()

In [ ]:
cnv.head()

### Check duplicate patient IDs

Are there duplicate column names (more than one aliquote from the same patient)? If so, keep only one after checking that all values are equal.

In [ ]:
# Any non-replaced column names
old_names = []
for col in cnv.columns:
    if not col.startswith('TCGA-'):
        old_names.append(col)

print('# columns with old name (not replaced by patient ID):', len(old_names))

In [ ]:
# Non-unique column names
len(cnv.columns) - len(set(cnv.columns))

In [ ]:
from collections import Counter

c = Counter(cnv.columns)

duplicated_cols = [col for col in c if c[col] > 1]
len(duplicated_cols)

In [ ]:
set(c.values())

In [ ]:
# Are repeats different?

not_equal_cols = []

for col in duplicated_cols:
    n_repeats = cnv.loc[:, col].shape[1]
    for i in range(n_repeats):
        if not cnv.loc[:, col].iloc[:, 0].equals(cnv.loc[:, col].iloc[:, 1]):
            not_equal_cols.append(col)

In [ ]:
len(set(not_equal_cols))

In [ ]:
not_equal_cols[:5]

In [ ]:
# How different are they? Look at a couple of examples
eg = cnv.loc[:, not_equal_cols[60]]
eg.head()

In [ ]:
print('# equal rows:', len(np.where(eg.iloc[:, 0] == eg.iloc[:, 1])[0]))
print('# different rows:', len(np.where(eg.iloc[:, 0] != eg.iloc[:, 1])[0]))

Variable number of differences. My non-systematic check returned values from as low as 5 different values to as high as 1'744. For now, just drop all but one column at random.

In [ ]:
cnv = cnv.T
cnv.head()

In [ ]:
cnv.shape

In [ ]:
# Non-unique column names
cnv[cnv.index.duplicated()].shape

In [ ]:
len(cnv)

In [ ]:
cnv = cnv.drop_duplicates(keep='first')
len(cnv)

In [ ]:
# Non-unique column names
cnv[cnv.index.duplicated()].shape

In [ ]:
cnv_copy = cnv.copy()

In [ ]:
cnv_copy.shape

In [ ]:
idxs = []
to_drop = []
n = cnv_copy.shape[0]

for i, idx in enumerate(cnv_copy.index):
    print('\r' + f'Check patient IDs for repeats: {str(i + 1)}/{n}', end='')
    if idx not in idxs:
        idxs.append(idx)
    else:
        to_drop.append(i)

In [ ]:
len(to_drop)

In [ ]:
cnv.drop(cnv.index[to_drop], inplace=True)

In [ ]:
cnv_copy.shape

In [ ]:
cnv.shape

## Encode categorical variables

Encode values (0, 1, or -1) to use with categorical embeddings.

In [ ]:
cnv.head()

In [ ]:
label_encoders = {}
for gene in cnv.columns:
    label_encoders[gene] = LabelEncoder()
    label_encoders[gene].fit_transform(cnv.loc[:, gene])

for gene in cnv.columns:
    cnv.loc[:, gene] = label_encoders[gene].fit_transform(cnv.loc[:, gene])

In [ ]:
pd.unique(cnv.iloc[:, 0])

In [ ]:
pd.unique([int(cnv[col].nunique()) for col in cnv.columns])

All genes have embedding dimension 3. According to the rule-of-thumb from the Fast.ai course the embedding size should be 2.

In [ ]:
cat_dim = 3
emb_size = min(50, (cat_dim + 1) // 2)
print(f'Embedding size of {emb_size}')
print(f'(categorical dimension {cat_dim} and a total of {cnv.shape[1]} features)')

### Drop unused patients

Keep only patients present in label data.

In [ ]:
labels = pd.read_csv('../data/labels.tsv', sep='\t')
len(labels['submitter_id'])

In [ ]:
cnv.shape

In [ ]:
len(cnv.index)

In [ ]:
%%time

for patient in cnv.index:
    if patient not in list(labels['submitter_id']):
        cnv.drop(patient, inplace=True)

In [ ]:
cnv.shape

## Keep only high-variance genes

In [ ]:
var = cnv.var()

In [ ]:
var[var > 1e-2].sort_values(ascending=False).plot(use_index=False)

In [ ]:
# Variance quantile to drop all but the chosen number of genes showing the top variance
q = (len(var) - 5e3) / len(var)
print(f'Percentile = {round(q * 100, 1)}%')

var[var > var.quantile(q=q)].shape

In [ ]:
# Variance quantile to drop all but the chosen number of genes showing the top variance
q = (len(var) - 2e3) / len(var)
print(f'Percentile = {round(q * 100, 1)}%')

var[var > var.quantile(q=q)].shape

In [ ]:
# Get IDs for all genes to keep
genes_to_keep = list(var[var > var.quantile(q=q)].index)
genes_to_keep[:5]

In [ ]:
len(genes_to_keep)

In [ ]:
cnv = cnv[genes_to_keep]

In [ ]:
cnv.shape

## Save to files

# Watermark <a class='tocSkip'></a>

In [ ]:
%watermark --iversions
%watermark -v
print()
%watermark -u -n

[Top of the page](#Top)